https://blog.csdn.net/program_developer/article/details/104263623

https://blog.csdn.net/weixin_42305672/article/details/111252715

In [1]:
import catboost as cb

In [ ]:
def cb_model(train, target, test, k):
    # saveFeature_df = pd.read_csv('../feature/xgb_920_556_5_score.csv',header=None)
#     saveFeature_df2 = pd.read_csv('../feature/xgb_09-20_74_0.8_0.6_5_score.csv',header=None)
    
    # saveFeature_df.columns=['feature','score']
#     saveFeature_df2.columns=['feature','score']

#     saveFeature_list = list(saveFeature_df['feature'].values)
#     saveFeature_list2 = list(saveFeature_df2['feature'].values)
    
    # saveFeature_list = list(saveFeature_df[saveFeature_df['score']>10]['feature'])
    saveFeature_list=list(train.columns)
    feats = [f for f in saveFeature_list if f not in ['id', 'isDefault']]
    feaNum = len(feats)
    print('Current num of features:', len(feats))

    seeds = [2020,666666,188888]
    output_preds = 0
    xgb_oof_probs = np.zeros(train.shape[0])

    for seed in seeds:
        folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
        oof_probs = np.zeros(train.shape[0])

        offline_score = []
        feature_importance_df = pd.DataFrame()
        params = {'booster': 'gbtree',
                  'objective': 'binary:logistic',
                  'eval_metric': 'auc',
                  'min_child_weight': 5,
                  'max_depth': 8,
                  'subsample': ss,
                  'colsample_bytree': fs,
                  'eta': 0.01,

                  'seed': seed,
                  'nthread': -1,

                  'tree_method': 'hist'
                  }
        for i, (train_index, test_index) in enumerate(folds.split(train, target)):
            
            train_y, test_y = target[train_index], target[test_index]
            train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]
            model = cb.CatBoostClassifier(n_estimators=1000,learning_rate = 0.1,max_depth=6,random_state=seed)
            model.fit(train_X,train_y)
            val_pred = model.predict(test_X)
            train_pred = model.predict(train_X)
            xgb_oof_probs[test_index] += val_pred / len(seeds)
            # oof_probs[test_index] += val_pred
            test_pred = model.predict(test)

            # 绘制roc曲线
            train_auc_value, valid_auc_value = plotroc(train_y, train_pred, test_y, val_pred)
            print('train_auc:{},valid_auc{}'.format(train_auc_value, valid_auc_value))
            offline_score.append(valid_auc_value)
            print(offline_score)
            output_preds += test_pred / k / len(seeds)
            
#             sub_df = test[['id']].copy()
#             sub_df['isDefault'] = output_preds
#             off = test[['id']].copy()
#             subVal_df = train[['id']].copy()
#             subVal_df.loc[test_index,'isDefault'] = xgb_oof_probs[test_index]
#             outpath = '../user_data/fold/'
#             fold_score = round(valid_auc_value, 5)
#             sub_df.to_csv( outpath + str(fold_score) + '_' + str(feaNum) + '_' + nowtime + '_{}_{}_xgb.csv'.format(i, kflod_num),
#             index=False)
#             subVal_df.to_csv(outpath + str(fold_score) + '_' + str(feaNum) + '_' + nowtime + '_{}_{}_xgbVal.csv'.format(i, kflod_num),
#             index=False)

        print('all_auc:', roc_auc_score(target.values, oof_probs))
        print('OOF-MEAN-AUC:%.6f, OOF-STD-AUC:%.6f' % (np.mean(offline_score), np.std(offline_score)))

    return output_preds, xgb_oof_probs, np.mean(offline_score), feaNum

In [25]:
# 特征工程

grade_dict = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6}

employmentLength_dict = {'1 year':1,'10+ years':10,'2 years':2,'3 years':3,'4 years':4,
                         '5 years':5,'6 years':6,'7 years':7,'8 years':8,'9 years':9,'< 1 year':0}

def get_sub_grade(grade, sub):
    return grade*10+int(sub[1])

def trans_issueDate(issueDate):
    year,month,day = issueDate.split('-')
    return int(year)*12+int(month)-1

def trans_earliesCreditLine(earliesCreditLine):
    month_dict = {"Jan":1, "Feb":2, "Mar":3, "Apr":4, "May":5, "Jun":6, "Jul":7, "Aug":8, "Sep":9, "Oct":10, "Nov":11, "Dec":12}
    month,year = earliesCreditLine.split('-')
    month = month_dict[month]
    return int(year)*12+month-1

# 类别特征
cate_features = ['employmentTitle', 'employmentLength_bin', 'purpose', 'postCode', 'subGrade', 'earliesCreditLine_bin', \
                 'regionCode', 'title', 'issueDate_bin', 'term_bin',\
                 'interestRate_bin', 'annualIncome_bin', 'loanAmnt_bin','homeOwnership_bin',\
                 'revolBal_bin','dti_bin','installment_bin','revolBal_bin','revolUtil_bin']

def process(dfs):
    for df in dfs:
        print(df.shape)
        df['grade'] = df['grade'].apply(lambda x: x if x not in grade_dict else grade_dict[x])
        df['subGrade'] = df.apply(lambda row: get_sub_grade(row['grade'],row['subGrade']), axis=1)
        df['employmentLength'] = df['employmentLength'].apply(lambda x: x if x not in employmentLength_dict else employmentLength_dict[x])
        #df['issueYear'] = df['issueDate'].apply(lambda x: int(x.split('-')[0]))
        df['issueDate'] = df['issueDate'].apply(lambda x: trans_issueDate(x))
        df['earliesCreditLine'] = df['earliesCreditLine'].apply(lambda x: trans_earliesCreditLine(x))
        df['date_Diff'] = df['issueDate'] - df['earliesCreditLine']
        df['installment_term_revolBal'] = df['installment']*12*df['term']/(df['revolBal']+0.1)
        df['revolUtil_revolBal'] = df['revolUtil']/(df['revolBal']+0.1)
        df['openAcc_totalAcc'] = df['openAcc']/df['totalAcc']
        df['dti'] = np.abs(df['dti'].fillna(1000))
        df['loanAmnt_dti_annualIncome'] = df['loanAmnt']/(np.abs(df['dti'])*df['annualIncome']+0.1)
        df['employmentLength_bin'] = df['employmentLength']
        df['issueDate_bin'] = df['issueDate']
        df['earliesCreditLine_bin'] = df['earliesCreditLine']
        df['term_bin'] = df['term']
        df['homeOwnership_bin'] = df['homeOwnership']
        df['annualIncome_loanAmnt'] = df['annualIncome']/(df['loanAmnt']+0.1)
        df['revolBal_loanAmnt'] = df['revolBal']/(df['loanAmnt']+0.1)
        df['revolBal_installment'] = df['revolBal']/(df['installment']+0.1)
        df['annualIncome_installment'] = df['annualIncome']/(df['installment']+0.1)
    concated_df = pd.concat(dfs)
    label_lst = []
    # 把分箱后的特征做为类别特征处理
    bin_number = 10
    for i in range(bin_number):
        label_lst.append(i)
    dfs[0]['annualIncome_bin'] = pd.qcut(concated_df['annualIncome'], bin_number, labels=label_lst,duplicates='drop')[:dfs[0].shape[0]]
    dfs[0]['loanAmnt_bin'] = pd.qcut(concated_df['loanAmnt'], bin_number, labels=label_lst,duplicates='drop')[:dfs[0].shape[0]]
    dfs[1]['annualIncome_bin'] = pd.qcut(concated_df['annualIncome'], bin_number, labels=label_lst,duplicates='drop')[dfs[0].shape[0]:]
    dfs[1]['loanAmnt_bin'] = pd.qcut(concated_df['loanAmnt'], bin_number, labels=label_lst,duplicates='drop')[dfs[0].shape[0]:]

    label_lst = []
    bin_number = 100
    for i in range(bin_number):
        label_lst.append(i)
    dfs[0]['interestRate_bin'] = pd.qcut(concated_df['revolBal'], bin_number, labels=label_lst,duplicates='drop')[:dfs[0].shape[0]]
    dfs[0]['dti_bin'] = pd.qcut(concated_df['dti'], bin_number, labels=label_lst,duplicates='drop')[:dfs[0].shape[0]]
    dfs[0]['installment_bin'] = pd.qcut(concated_df['installment'], bin_number, labels=label_lst,duplicates='drop')[:dfs[0].shape[0]]
    dfs[0]['revolBal_bin'] = pd.qcut(concated_df['revolBal'], bin_number, labels=label_lst,duplicates='drop')[:dfs[0].shape[0]]
    dfs[0]['revolUtil_bin'] = pd.qcut(concated_df['revolUtil'], bin_number, labels=label_lst,duplicates='drop')[:dfs[0].shape[0]]

    dfs[1]['interestRate_bin'] = pd.qcut(concated_df['revolBal'], bin_number, labels=label_lst,duplicates='drop')[dfs[0].shape[0]:]
    dfs[1]['dti_bin'] = pd.qcut(concated_df['dti'], bin_number, labels=label_lst,duplicates='drop')[dfs[0].shape[0]:]
    dfs[1]['installment_bin'] = pd.qcut(concated_df['installment'], bin_number, labels=label_lst,duplicates='drop')[dfs[0].shape[0]:]
    dfs[1]['revolBal_bin'] = pd.qcut(concated_df['revolBal'], bin_number, labels=label_lst,duplicates='drop')[dfs[0].shape[0]:]
    dfs[1]['revolUtil_bin'] = pd.qcut(concated_df['revolUtil'], bin_number, labels=label_lst,duplicates='drop')[dfs[0].shape[0]:]

    for df in dfs:
        for cate in cate_features:
            df[cate] = df[cate].fillna(0).astype('int')
    issueDate_lst = list(set(concated_df['issueDate']))
    ratio_feat_lst = ['loanAmnt', 'installment', 'interestRate', 'annualIncome', 'dti', 'openAcc', \
                      'revolBal', 'revolUtil', 'totalAcc']
    employmentLength_lst = list(set(concated_df['employmentLength']))
    purpose_lst = list(set(concated_df['purpose']))
    homeOwnership_lst = list(set(concated_df['homeOwnership']))
    for feat in ratio_feat_lst:
        issueDate_median = {}
        issueDate_item_rank = {}
        issueDate_label_mean = {}
        for dt in issueDate_lst:
            # 取最近6个月
            mask = (concated_df['issueDate'] >= dt-3)&(concated_df['issueDate'] <= dt+3)
            # 取最近6个月除去当前月份
            mask_1 = (concated_df['issueDate'] >= dt-3)&(concated_df['issueDate'] <= dt+3)&(concated_df['issueDate'] != dt)
            item_series = concated_df.loc[mask, feat]
            label_series = concated_df.loc[mask_1, 'isDefault']
            # 取最近6个月的中位数
            issueDate_median[dt] = item_series.median()
            issueDate_label_mean[dt] = label_series.mean()
            item_rank = item_series.rank()/len(item_series)
            issueDate_item_rank[dt] = {}
            for item,rank in zip(item_series, item_rank):
                issueDate_item_rank[dt][item] = rank
        employmentLength_median = {}
        for et in employmentLength_lst:
            mask = concated_df['employmentLength'] == et
            item_series = concated_df.loc[mask, feat]
            employmentLength_median[et] = item_series.median()
        purpose_median = {}
        for pp in purpose_lst:
            mask = concated_df['purpose'] == pp
            item_series = concated_df.loc[mask, feat]
            purpose_median[pp] = item_series.median()
        homeOwnership_median = {}
        for ho in homeOwnership_lst:
            mask = concated_df['homeOwnership'] == ho
            item_series = concated_df.loc[mask, feat]
            homeOwnership_median[ho] = item_series.median()
        for df in dfs:
            print(feat, df.shape)
            df['label_issueDate_mean'] = df['issueDate'].apply(lambda x: issueDate_label_mean[x])
            df[feat+'_issueDate_median'] = df['issueDate'].apply(lambda x: issueDate_median[x])
            #df['interestRate_ratio'] = df['interestRate']/df['interestRate_median']
            df[feat+'_issueDate_ratio'] = df.fillna(0).apply(lambda r: issueDate_item_rank[r['issueDate']][r[feat]], axis=1)
            df[feat+'_employmentLength_ratio'] = df.fillna(0).apply(lambda r: r[feat]/employmentLength_median[r['employmentLength']], axis=1)
            df[feat+'_purpose_ratio'] = df.fillna(0).apply(lambda r: r[feat]/purpose_median[r['purpose']], axis=1)
            df[feat+'_homeOwnership_ratio'] = df.fillna(0).apply(lambda r: r[feat]/homeOwnership_median[r['homeOwnership']], axis=1)
            print(feat, df.shape)
        return dfs 

# train_data = pd.read_csv('train.csv')
# test_data = pd.read_csv('testA.csv')

a = process([train_data, test_data])

(10000, 114)


<ipython-input-25-45a50dff398a>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['grade'] = df['grade'].apply(lambda x: x if x not in grade_dict else grade_dict[x])


IndexError: invalid index to scalar variable.

In [1]:
import pandas as pd
import numpy as np

In [30]:
train = pd.read_csv('/Users/senlinlidewo/pyprogram/天池竞赛-零基础金融风控/数据/train.csv')
train_data = train[:10000]

In [31]:
test = pd.read_csv('/Users/senlinlidewo/pyprogram/天池竞赛-零基础金融风控/数据/testA.csv')
test_data = test[:5000]

In [32]:
combine = pd.concat([train_data,test_data])
combine

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,0,35000.0,5,19.52,917.97,E,E2,320.0,2 years,2,...,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0
1,1,18000.0,5,18.49,461.90,D,D2,219843.0,5 years,0,...,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN
2,2,12000.0,5,16.99,298.17,D,D3,31698.0,8 years,0,...,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0
3,3,11000.0,3,7.26,340.96,A,A4,46854.0,10+ years,1,...,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0
4,4,3000.0,3,12.99,101.07,C,C2,54.0,NaN,1,...,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,804995,18725.0,5,20.80,504.47,E,E1,231898.0,8 years,1,...,3.0,12.0,10.0,14.0,6.0,14.0,0.0,0.0,0.0,2.0
4996,804996,12000.0,3,21.97,458.10,E,E4,163043.0,6 years,1,...,8.0,3.0,7.0,19.0,6.0,12.0,NaN,0.0,0.0,2.0
4997,804997,8400.0,3,5.32,252.97,A,A1,16598.0,< 1 year,1,...,9.0,3.0,10.0,10.0,3.0,11.0,0.0,0.0,0.0,0.0
4998,804998,10200.0,3,13.49,346.10,C,C2,38.0,10+ years,0,...,7.0,5.0,9.0,12.0,7.0,13.0,0.0,0.0,0.0,5.0


In [13]:
def missing_percentage(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100,2)
    return pd.concat([total, percent], axis=1, keys=['Total','Percent'])
missing_value_percent = missing_percentage(combine)
missing_value_percent

,Total,Percent
isDefault,5000,33.33
n11,1352,9.01
employmentLength,854,5.69
n14,773,5.15
n6,773,5.15
n1,773,5.15
n2,773,5.15
n3,773,5.15
n5,773,5.15
n7,773,5.15


In [14]:
cols = missing_value_percent.index[missing_value_percent.Percent>0]
list(cols)

['isDefault',
 'n11',
 'employmentLength',
 'n14',
 'n6',
 'n1',
 'n2',
 'n3',
 'n5',
 'n7',
 'n8',
 'n9',
 'n12',
 'n13',
 'n0',
 'n4',
 'n10',
 'revolUtil',
 'pubRecBankruptcies',
 'dti']

In [15]:
cols = [f for f in cols if f not in ['employmentLength','isDefault']]
for col in cols:
    combine[col].fillna(r'\N', inplace=True)
for col in cols:
    combine[col].replace({r'\N': -999}, inplace=True)
    combine[col] = combine[col]

In [16]:
missing_value_percent = missing_percentage(combine)
missing_value_percent

,Total,Percent
isDefault,5000,33.33
employmentLength,854,5.69
id,0,0.00
initialListStatus,0,0.00
applicationType,0,0.00
earliesCreditLine,0,0.00
title,0,0.00
policyCode,0,0.00
n0,0,0.00
n1,0,0.00


In [17]:
combine.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,0,35000.0,5,19.52,917.97,E,E2,320.0,2 years,2,...,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0
1,1,18000.0,5,18.49,461.90,D,D2,219843.0,5 years,0,...,-999.0,-999.0,-999.0,-999.0,-999.0,13.0,-999.0,-999.0,-999.0,-999.0
2,2,12000.0,5,16.99,298.17,D,D3,31698.0,8 years,0,...,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0
3,3,11000.0,3,7.26,340.96,A,A4,46854.0,10+ years,1,...,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0
4,4,3000.0,3,12.99,101.07,C,C2,54.0,NaN,1,...,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0


In [18]:
####特征工程####

In [19]:
grade_dict = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6}

employmentLength_dict = {'1 year':1,'10+ years':10,'2 years':2,'3 years':3,'4 years':4,
                         '5 years':5,'6 years':6,'7 years':7,'8 years':8,'9 years':9,'< 1 year':0}
df['grade'] = df['grade'].apply(lambda x: x if x not in grade_dict else grade_dict[x])
df['subGrade'] = df.apply(lambda row: get_sub_grade(row['grade'],row['subGrade']), axis=1)

In [33]:
train_data

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,0,35000.0,5,19.52,917.97,E,E2,320.0,2 years,2,...,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0
1,1,18000.0,5,18.49,461.90,D,D2,219843.0,5 years,0,...,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN
2,2,12000.0,5,16.99,298.17,D,D3,31698.0,8 years,0,...,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0
3,3,11000.0,3,7.26,340.96,A,A4,46854.0,10+ years,1,...,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0
4,4,3000.0,3,12.99,101.07,C,C2,54.0,NaN,1,...,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,16000.0,3,13.98,546.69,C,C3,639.0,1 year,1,...,5.0,5.0,6.0,10.0,4.0,10.0,0.0,0.0,0.0,3.0
9996,9996,16000.0,5,16.78,395.76,C,C5,288216.0,2 years,0,...,9.0,31.0,9.0,15.0,9.0,14.0,0.0,0.0,0.0,2.0
9997,9997,11200.0,5,15.99,272.31,C,C5,9052.0,2 years,0,...,3.0,1.0,2.0,5.0,2.0,4.0,0.0,0.0,0.0,1.0
9998,9998,16000.0,5,15.58,385.53,D,D3,220660.0,5 years,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
label_lst = []
bin_number = 10
for i in range(bin_number):
    label_lst.append(i)
combine['annualIncome_bin'] = pd.qcut(combine['annualIncome'], bin_number, labels=label_lst,duplicates='drop')[:combine.shape[0]]
combine


,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n6,n7,n8,n9,n10,n11,n12,n13,n14,annualIncome_bin
0,0,35000.0,5,19.52,917.97,E,E2,320.0,2 years,2,...,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,8
1,1,18000.0,5,18.49,461.90,D,D2,219843.0,5 years,0,...,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,2
2,2,12000.0,5,16.99,298.17,D,D3,31698.0,8 years,0,...,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,5
3,3,11000.0,3,7.26,340.96,A,A4,46854.0,10+ years,1,...,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0,8
4,4,3000.0,3,12.99,101.07,C,C2,54.0,NaN,1,...,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,804995,18725.0,5,20.80,504.47,E,E1,231898.0,8 years,1,...,12.0,10.0,14.0,6.0,14.0,0.0,0.0,0.0,2.0,2
4996,804996,12000.0,3,21.97,458.10,E,E4,163043.0,6 years,1,...,3.0,7.0,19.0,6.0,12.0,NaN,0.0,0.0,2.0,3
4997,804997,8400.0,3,5.32,252.97,A,A1,16598.0,< 1 year,1,...,3.0,10.0,10.0,3.0,11.0,0.0,0.0,0.0,0.0,9
4998,804998,10200.0,3,13.49,346.10,C,C2,38.0,10+ years,0,...,5.0,9.0,12.0,7.0,13.0,0.0,0.0,0.0,5.0,7


In [43]:
issueDate_lst = list(set(combine['issueDate']))
issueDate_lst

['2016-02-01',
 '2010-04-01',
 '2014-03-01',
 '2015-06-01',
 '2008-10-01',
 '2017-07-01',
 '2014-10-01',
 '2014-01-01',
 '2017-01-01',
 '2011-08-01',
 '2014-07-01',
 '2015-10-01',
 '2015-07-01',
 '2017-02-01',
 '2012-07-01',
 '2016-12-01',
 '2014-02-01',
 '2014-05-01',
 '2010-11-01',
 '2015-01-01',
 '2011-11-01',
 '2008-07-01',
 '2017-05-01',
 '2017-08-01',
 '2008-11-01',
 '2017-11-01',
 '2018-09-01',
 '2012-10-01',
 '2013-12-01',
 '2013-01-01',
 '2013-03-01',
 '2009-07-01',
 '2017-09-01',
 '2015-03-01',
 '2016-08-01',
 '2007-12-01',
 '2016-09-01',
 '2011-01-01',
 '2011-09-01',
 '2015-12-01',
 '2015-09-01',
 '2016-04-01',
 '2013-09-01',
 '2011-04-01',
 '2012-08-01',
 '2016-07-01',
 '2018-10-01',
 '2018-03-01',
 '2010-06-01',
 '2008-09-01',
 '2015-08-01',
 '2018-04-01',
 '2009-10-01',
 '2012-11-01',
 '2013-08-01',
 '2012-06-01',
 '2017-10-01',
 '2018-12-01',
 '2014-12-01',
 '2010-08-01',
 '2014-11-01',
 '2016-03-01',
 '2018-06-01',
 '2018-01-01',
 '2016-10-01',
 '2008-03-01',
 '2009-05-

In [29]:
list(train_data.columns)

['id',
 'loanAmnt',
 'term',
 'interestRate',
 'installment',
 'grade',
 'subGrade',
 'employmentTitle',
 'employmentLength',
 'homeOwnership',
 'annualIncome',
 'verificationStatus',
 'issueDate',
 'isDefault',
 'purpose',
 'postCode',
 'regionCode',
 'dti',
 'delinquency_2years',
 'ficoRangeLow',
 'ficoRangeHigh',
 'openAcc',
 'pubRec',
 'pubRecBankruptcies',
 'revolBal',
 'revolUtil',
 'totalAcc',
 'initialListStatus',
 'applicationType',
 'earliesCreditLine',
 'title',
 'policyCode',
 'n0',
 'n1',
 'n2',
 'n3',
 'n4',
 'n5',
 'n6',
 'n7',
 'n8',
 'n9',
 'n10',
 'n11',
 'n12',
 'n13',
 'n14',
 'date_Diff',
 'installment_term_revolBal',
 'revolUtil_revolBal',
 'openAcc_totalAcc',
 'loanAmnt_dti_annualIncome',
 'employmentLength_bin',
 'issueDate_bin',
 'earliesCreditLine_bin',
 'term_bin',
 'homeOwnership_bin',
 'annualIncome_loanAmnt',
 'revolBal_loanAmnt',
 'revolBal_installment',
 'annualIncome_installment',
 'annualIncome_bin',
 'loanAmnt_bin',
 'interestRate_bin',
 'dti_bin',
 '

In [45]:
employmentLength_lst = list(set(combine['employmentLength']))
employmentLength_lst

[nan,
 '9 years',
 '10+ years',
 '6 years',
 '< 1 year',
 '4 years',
 '1 year',
 '3 years',
 '2 years',
 '5 years',
 '7 years',
 '8 years']